In [ ]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import numpy as np
import torch
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt

In [100]:
!wget https://raw.githubusercontent.com/mohammedelabbas/SVM/main/Preprocessing.py
!wget https://raw.githubusercontent.com/mohammedelabbas/SVM/main/svm.py
!wget https://raw.githubusercontent.com/mohammedelabbas/SVM/main/word_representation.py

--2022-04-22 08:14:07--  https://raw.githubusercontent.com/mohammedelabbas/SVM/main/Preprocessing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1270 (1.2K) [text/plain]
Saving to: ‘Preprocessing.py’

Preprocessing.py    100%[===================>]   1.24K  --.-KB/s    in 0s      

2022-04-22 08:14:07 (91.9 MB/s) - ‘Preprocessing.py’ saved [1270/1270]

--2022-04-22 08:14:08--  https://raw.githubusercontent.com/mohammedelabbas/SVM/main/svm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1239 (1.2K) [text/plain]
Saving to: ‘sv

In [58]:
df = pd.read_csv("IMDB Dataset.csv")
df["review"][10]


'Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines.<br /><br />At first it was very odd and pretty funny but as the movie progressed I didn\'t find the jokes or oddness funny anymore.<br /><br />Its a low budget film (thats never a problem in itself), there were some pretty interesting characters, but eventually I just lost interest.<br /><br />I imagine this film would appeal to a stoner who is currently partaking.<br /><br />For something similar but better try "Brother from another planet"'

In [111]:
df["sentiment"] [df["sentiment"] == "positive"] = 1
df["sentiment"] [df["sentiment"] == "negative"] = -1
labels  = df["sentiment"]
labels = np.array(labels)

In [ ]:
from nltk.stem import PorterStemmer
import re
class DataPreprocessing:
  
  def _remove_html(self, data):
    for i in range(len(data)) :
      p = re.compile(r'<.*?>')
      data[i] = p.sub('', data[i])
      
    return data

  #lowercase
  def _convert_to_lower_case(self,data):
    for i in range(len(data)):
      data[i] = data[i].lower()
    return data  

  #single characters
  def _remove_single_characters(self, data):
    for i in range(len(data)) :  
      data[i] = " ".join([w for w in data[i].split() if len(w)>1]) 
    return data 

   #punctuation  
  def _remove_punctuation(self,data):
    symbols = "!,\"#$%&()*+-./:;<=>?@[\]^_`{|}~'\n"
    for symbol in symbols:
      for i in range(len(data)):
        data[i] = data[i].replace(symbol,"")
    return data  

  #stemming  
  def _stem(self,data):
    ps = PorterStemmer ()
    
    for i in range(len(data)) :
      new_doc = ""
      for word in data[i].split():
        new_doc+= ps.stem(word)+" "
      data[i] = new_doc  
    return data   
  
  def preprocess(self,data):
     data = self._remove_html(data)
     data = self._remove_single_characters(data)
     data = self._convert_to_lower_case(data)
     data = self._remove_punctuation(data)
     data = self._stem(data)

     return data

# Data Pre-processing

In [ ]:
import re

#from Preprocessing import DataPreprocessing
preprocessing = DataPreprocessing()
data = preprocessing.preprocess(list(df["review"]))

# Word Representation

**Count Frequency Encoding**

In [ ]:


import numpy as np

class WordFrequency:
  def _get_word_set(self,text): 
    sentences = []
    word_set = []
    for sent in text:
        x = [i.lower() for  i in sent.split() if i.isalpha()]
        sentences.append(x)
        for word in x:
            if word not in word_set:
                word_set.append(word)
    return set(word_set),sentences
  def _count_word(self,word,sentence):
    count = 0
    for w in sentence:
      if w==word:
        count +=1
    return count    

  def word_frequency(self,text):
    word_set , sentences = self._get_word_set(text) 
    print (sorted(word_set))
    word_count = []
    for sentence in sentences:
      row =[]
      for word in sorted(word_set):
        row.append(self._count_word(word,sentence))
      word_count.append(row)
    return np.array(word_count)

  

        



    
    

class WordRepresentation:
  def __init__(self, method = 'tf-idf'):
    self.method = method
  def count_frequency(self,text):
      wf = WordFrequency()
      return wf.word_frequency(text)

  def tf_idf(self,text):
    return TFIDF().tf_idf(text)     
  def get_representation(self,text):
    if self.method == "tf-idf":
      return self.tf_idf(text)
      
    elif self.method=="frequency-count":
      return self.count_frequency(text)
    else:
      raise Exception("Unknown method")
    pass



**TF-IDF**

In [ ]:
class TFIDF:

  def _get_word_set(self,text): 
    sentences = []
    word_set = []
    for sent in text:
        x = [i.lower() for  i in sent.split() if i.isalpha()]
        sentences.append(x)
        for word in x:
            if word not in word_set:
                word_set.append(word)
    return set(word_set),sentences          

  

  def _count_dict(self,sentences,word_set):
      word_count = {}
      for word in word_set:
          word_count[word] = 0
          for sent in sentences:
              if word in sent:
                  word_count[word] += 1
      return word_count
  
  #word_count = count_dict(sentences)    
  #Term Frequency
  def _termfreq(self,document, word):
      N = len(document)
      occurance = len([token for token in document if token == word])
      return occurance/N


  def _inverse_doc_freq(self,word,word_count,total_documents):
      try:
          word_occurance = word_count[word] + 1
      except:
          word_occurance = 1
      return np.log(total_documents/word_occurance)

  def tf_idf(self,text):
    word_set,sentences = self._get_word_set(text)
    #print (word_set)
    total_documents = len(sentences)
    #Creating an index for each word in our vocab.
    index_dict = {} #Dictionary to store index for each word
    i = 0
    for word in word_set:
      index_dict[word] = i
      i += 1
    #print (index_dict)  
    
    word_count = self._count_dict(sentences,word_set)
    print (len(word_count))
    vectors = []
    print (len(sentences))
    for sentence in sentences:
      tf_idf_vec = np.zeros((len(word_set),))
      for word in sentence:
          #print (word)
          tf = self._termfreq(sentence,word)
          idf = self._inverse_doc_freq(word,word_count,total_documents)
          value = tf*idf
          tf_idf_vec[index_dict[word]] = value 
      vectors.append(tf_idf_vec)    
    return np.array(vectors)   



In [ ]:
#from word_representation import WordRepresentation
repr = WordRepresentation() #method = "frequency-count"
x= repr.tf_idf(data)
x.shape

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,labels[:100], test_size = 0.1, random_state = 10)
x_train = x_train.astype(np.double)
y_train = y_train.astype(np.double)
x_test = x_test.astype(np.double)
y_test = y_test.astype(np.double)
print (x_train.shape)

#Training

In [126]:
!pip install cvxopt
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers
!pwd

/content


In [ ]:
#!pip install cvxopt
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers
import numpy as np

class SVM:
  def __init__(self):
    self.w = None
    self.b = None
  def train(self,X,y):
    m,n = X.shape
    y = y.reshape(-1,1) * 1.
    X_dash = y * X
    H = np.dot(X_dash , X_dash.T) * 1

    #Converting into cvxopt format
    P = cvxopt_matrix(H)
    q = cvxopt_matrix(-np.ones((m, 1)))
    G = cvxopt_matrix(-np.eye(m))
    h = cvxopt_matrix(np.zeros(m))
    A = cvxopt_matrix(y.reshape(1, -1))
    b = cvxopt_matrix(np.zeros(1))
    solution = solvers.qp(P, q, G, h, A, b)
    alpha = np.array(solution['x'])
    w = ((alpha * y).T @X).reshape(-1, 1)
    seuil = 0.0
    s = (alpha > seuil).flatten()
    b = y[s] - np.dot(X[s], w)
    self.w = w
    self.b =b
    return w, b 
  def test(self,X):
    print (X.shape)
    n = X.shape[0]
    y_pred = np.zeros((n,1))

    for i in range(n):
      distance=np.dot(np.array(X[i]), self.w) + self.b
    #print (distance.shape)
      y_pred[i] = np.sign(distance)[0]
    return y_pred

  def accuracy(self,y_pred,y):
      n= y_pred.shape[0]
      good_pred = 0.0
      for i in range(n):
        if y_pred[i] == y[i]:
          good_pred +=1
      return good_pred/n    

In [ ]:
#from svm import SVM
model = SVM()
y = model.train(x_train, y_train)



#Testing

In [133]:
y_pred = model.test(x_test)
model.accuracy(y_pred,y_test)


(10, 3980)


0.7

#Visualization

In [130]:
plt.figure(figsize=(10, 8))
# Plotting our two-features-space
sns.scatterplot(x=X_train[:, 0], 
                y=X_train[:, 1], 
                hue=y_train, 
                s=8);
# Constructing a hyperplane using a formula.
w = svc_model.coe          # w consists of 2 elements
b = svc_model.intercept_[0]      # b consists of 1 element
x_points = np.linspace(-1, 1)    # generating x-points from -1 to 1
y_points = -(w[0] / w[1]) * x_points - b / w[1]  # getting corresponding y-points
# Plotting a red hyperplane
plt.plot(x_points, y_points, c='r');

# Encircle support vectors
# plt.scatter(svc_model.support_vectors_[:, 0],
#             svc_model.support_vectors_[:, 1], 
#             s=50, 
#             facecolors='none', 
#             edgecolors='k', 
#             alpha=.5);

# Step 2 (unit-vector):
w_hat = Svmoptim_[0] / (np.sqrt(np.sum(SvmOptim_[0] ** 2)))
# Step 3 (margin):
margin = 1 / np.sqrt(np.sum(svc_model.coef_[0] ** 2))
# Step 4 (calculate points of the margin lines):
decision_boundary_points = np.array(list(zip(x_points, y_points)))
points_of_line_above = decision_boundary_points + w_hat * margin
points_of_line_below = decision_boundary_points - w_hat * margin
# Plot margin lines
# Blue margin line above
plt.plot(points_of_line_above[:, 0], 
         points_of_line_above[:, 1], 
         'b--', 
         linewidth=2)
# Green margin line below
plt.plot(points_of_line_below[:, 0], 
         points_of_line_below[:, 1], 
         'g--',
         linewidth=2)

NameError: ignored

<Figure size 720x576 with 0 Axes>